# Analiza czerniaka za pomocą fraktalnej sieci neuronowej

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Sprawdzamy dostępne urządzenie

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Zapisujemy konfigurację do zmiennych.

In [3]:
IMAGE_SIZE = 224
BATCH_SIZE = 32

Definiujemy warstwę, która będzie tworzyła obraz z fraktalnych cech podanych jej obrazów.

In [4]:
class Fractal2D(tf.keras.layers.Layer):
    def __init__(self, kernel_size_range):
        super(Fractal2D, self).__init__()
        self.kernel_size_range = kernel_size_range
    
    @staticmethod
    def binarize(patch_batch, kernel_size, batch_size):
        auxiliary_patch_batch = tf.image.resize_with_crop_or_pad(patch_batch, 1, 1)
        
        cd_binary = tf.cast(tf.math.less_equal(tf.math.reduce_max(tf.math.abs(tf.math.subtract(patch_batch, auxiliary_patch_batch)), axis=3), kernel_size), tf.int32)
        ed_binary = tf.cast(tf.math.less_equal(tf.math.pow(tf.math.reduce_sum(tf.math.pow(tf.math.subtract(patch_batch, auxiliary_patch_batch), 2), axis=3), 0.5), kernel_size), tf.int32)
        md_binary = tf.cast(tf.math.less_equal(tf.math.reduce_sum(tf.math.abs(tf.math.subtract(patch_batch, auxiliary_patch_batch)), axis=3), kernel_size), tf.int32)
        
        return  tf.reshape(cd_binary, shape=(batch_size, -1, kernel_size, kernel_size)), \
                tf.reshape(ed_binary, shape=(batch_size, -1, kernel_size, kernel_size)), \
                tf.reshape(md_binary, shape=(batch_size, -1, kernel_size, kernel_size))
    
    @staticmethod
    def calculate_probability_matrix(binary_matrix):
        # tf.map_fn(lambda image: tf.map_fn(lambda patch: tf.math.reduce_sum(patch), image), binary_matrix)
        pass
            
            
    def call(self, image_batch):
        for kernel_size in range(self.kernel_size_range[0], self.kernel_size_range[1] + 1, 2):
            patch_batch = tf.image.extract_patches(image_batch,
                                                     sizes=(1, kernel_size, kernel_size, 1),
                                                     strides=(1, kernel_size, kernel_size, 1),
                                                     rates=(1, 1, 1, 1),
                                                     padding='SAME')
            batch_size, row_size, col_size, depth = patch_batch.shape
            patch_batch = tf.reshape(patch_batch, shape=(batch_size * row_size * col_size, kernel_size, kernel_size, 3))

            cd_binary, ed_binary, md_binary = self.binarize(patch_batch, kernel_size, batch_size)
        return image_batch

Ładujemy dane do trenowania i walidacji.

In [5]:
datagen = ImageDataGenerator(validation_split=0.2, rescale=1.0)
training_set = datagen.flow_from_directory('/small-data',
                                           target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                           batch_size=BATCH_SIZE,
                                           class_mode='categorical',
                                           subset='training')
validation_set = datagen.flow_from_directory('/small-data',
                                             target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                             batch_size=BATCH_SIZE,
                                             class_mode='categorical',
                                             subset='validation')

Found 640 images belonging to 3 classes.
Found 159 images belonging to 3 classes.


Zapisujemy ilość rozpoznawalnych diagnoz.

In [6]:
DIAGNOSIS_NUMBER = len(training_set.class_indices)

Tworzymy model, który wykorzystuje wcześniej zdefiniowaną warstwę.

In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224, 224, 3), batch_size=32),
    Fractal2D(kernel_size_range=(3, 41)),
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4", output_shape=[1280],
                   trainable=False),
    tf.keras.layers.Dense(DIAGNOSIS_NUMBER, activation='softmax')
])

cd_binary: (32, 5625, 3, 3)
ed_binary: (32, 5625, 3, 3)
md_binary: (32, 5625, 3, 3)
cd_binary: (32, 2025, 5, 5)
ed_binary: (32, 2025, 5, 5)
md_binary: (32, 2025, 5, 5)
cd_binary: (32, 1024, 7, 7)
ed_binary: (32, 1024, 7, 7)
md_binary: (32, 1024, 7, 7)
cd_binary: (32, 625, 9, 9)
ed_binary: (32, 625, 9, 9)
md_binary: (32, 625, 9, 9)
cd_binary: (32, 441, 11, 11)
ed_binary: (32, 441, 11, 11)
md_binary: (32, 441, 11, 11)
cd_binary: (32, 324, 13, 13)
ed_binary: (32, 324, 13, 13)
md_binary: (32, 324, 13, 13)
cd_binary: (32, 225, 15, 15)
ed_binary: (32, 225, 15, 15)
md_binary: (32, 225, 15, 15)
cd_binary: (32, 196, 17, 17)
ed_binary: (32, 196, 17, 17)
md_binary: (32, 196, 17, 17)
cd_binary: (32, 144, 19, 19)
ed_binary: (32, 144, 19, 19)
md_binary: (32, 144, 19, 19)
cd_binary: (32, 121, 21, 21)
ed_binary: (32, 121, 21, 21)
md_binary: (32, 121, 21, 21)
cd_binary: (32, 100, 23, 23)
ed_binary: (32, 100, 23, 23)
md_binary: (32, 100, 23, 23)
cd_binary: (32, 81, 25, 25)
ed_binary: (32, 81, 25, 25)
md

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.fit(training_set, validation_data=validation_set, epochs=20)

Epoch 1/20
cd_binary: (32, 5625, 3, 3)
ed_binary: (32, 5625, 3, 3)
md_binary: (32, 5625, 3, 3)
cd_binary: (32, 2025, 5, 5)
ed_binary: (32, 2025, 5, 5)
md_binary: (32, 2025, 5, 5)
cd_binary: (32, 1024, 7, 7)
ed_binary: (32, 1024, 7, 7)
md_binary: (32, 1024, 7, 7)
cd_binary: (32, 625, 9, 9)
ed_binary: (32, 625, 9, 9)
md_binary: (32, 625, 9, 9)
cd_binary: (32, 441, 11, 11)
ed_binary: (32, 441, 11, 11)
md_binary: (32, 441, 11, 11)
cd_binary: (32, 324, 13, 13)
ed_binary: (32, 324, 13, 13)
md_binary: (32, 324, 13, 13)
cd_binary: (32, 225, 15, 15)
ed_binary: (32, 225, 15, 15)
md_binary: (32, 225, 15, 15)
cd_binary: (32, 196, 17, 17)
ed_binary: (32, 196, 17, 17)
md_binary: (32, 196, 17, 17)
cd_binary: (32, 144, 19, 19)
ed_binary: (32, 144, 19, 19)
md_binary: (32, 144, 19, 19)
cd_binary: (32, 121, 21, 21)
ed_binary: (32, 121, 21, 21)
md_binary: (32, 121, 21, 21)
cd_binary: (32, 100, 23, 23)
ed_binary: (32, 100, 23, 23)
md_binary: (32, 100, 23, 23)
cd_binary: (32, 81, 25, 25)
ed_binary: (32, 81,

KeyboardInterrupt: 